# Логгирование экспериментов
Для исследователей крайне важно содержать в чистоте результаты экспериментов, ведь в случаях поиска архитектур, гиперпараметров и т.д. может быть воспроизведено большое множество экспериментов. В случае если число проведенных экспериментов переваливает за десятки - становится крайне сложно следить за результатами и производить перекрестное сравнение, находить ошибки, выбирать лучший вариант. 

Для борьбы с выше перечисленными проблемами созданы инструменты логгирования и агрегирования экспериментов такие как:
 * [`weights and biases`](https://wandb.ai/site)
 * [`mlflow`](https://mlflow.org/)
 * [`comet.ml`](https://www.comet.ml/)
 * [`Tensorboard`](https://www.tensorflow.org/tensorboard)
 
Большинство из данынх инструментов распространяются на платной основе. Однако чаще всего студенты могут получит полный доступ к инструменту зарегестрировавшись по почте с доменом университета или используя github аккаунт подтвержденый в github-students.

В настоящем руководстве будет рассмотрен только инструмент `comet.ml`. Он полностью бесплатен в случае если вы собираете использовать его функционал единолично.

Для использования `comet.ml` необходимо установить пакет `comet-ml`

`pip install comet-ml==3.26.1`

## Comet.ml в `Pytorch`
Возьмем в качестве исходного кода - код используемый в файле `pytorch_basics`

In [1]:
from comet_ml import Experiment

In [22]:
import torch
import torch.optim as optim
import torch.nn as nn
import torchvision
from torchvision import transforms

import numpy as np
import matplotlib.pyplot as plt

import datetime
from tqdm import tqdm

device = 'cuda:1'

Создадим объект эксперимента.

Для его создания нужно иметь `API_KEY` который **можно получить в ЛК comet.ml**

Данный объект имеет большой набор параметров, о которых вы можете узнать из документации однако нас устроят и стандартные значения параметров

In [ ]:
experiment = Experiment(
    api_key='***',
    project_name='pytorch_cometml_lightning',
)

### Назавние эксперимента и теги

In [7]:
experiment.set_name("Testing comet.ml logging")
experiment.add_tag("Test")

Создав объект эксперимента он сразу появится в comet.ml в соответствующем проекте. 

![Overview](./imgs/overivew.png)

### Логгирование гиперпараметров

In [8]:
hyperparameters = {
    "learning_rate": 0.001,
    "batch_size": 1024,
    "num_workers": 4,
    "in_channels": 1
}

In [9]:
experiment.log_parameters(hyperparameters)

Выполнив метод выше мы можем увидеть, что переданные гиперпараметры сохранились в `comet.ml`

![Hyperparameters](./imgs/hyperparameters_logging.png)

### Логгирование метрик

In [12]:
def training_loop(n_epochs, optimizer, model, loss_fn, dataloader, experiment: Experiment):
    step = 0
    for epoch in range(1, n_epochs + 1):
        # Откроем контекст в котором производится шаг обучения сети
        # Это опционально
        with experiment.train():
            loss_train = 0.0
            model.train()
            for X, y in tqdm(dataloader):
                X = X.to(device=device)
                y = y.to(device=device)

                optimizer.zero_grad()
                pred = model(X)
                loss = loss_fn(pred, y)

                loss.backward()
                optimizer.step()
                
                # Добавим логгирование значений лосса
                experiment.log_metric("TrainLoss", loss.item(), step=step, epoch=epoch)
                step += 1
                loss_train += loss.item()
        
        # Тестирование модели
        if epoch == 1 or epoch % 2 == 0:
            with experiment.test():
                model.eval()
                with torch.no_grad():
                    size = len(dataloader.dataset)
                    correct = 0
                    for X, y in dataloader:
                        X = X.to(device=device)
                        y = y.to(device=device)
                        pred = model(X)
                        correct += (pred.argmax(dim=1) == y).type(torch.float).sum().item()
                    acc = correct / size
                
                # Логирование значений точности
                experiment.log_metric('Accuracy', acc, epoch=epoch)
                print('{} Epoch {}, Training loss {}, Accuracy {}'.format(
                    datetime.datetime.now(), epoch,
                    loss_train / len(dataloader), acc))

### Подготовим все необходимое для обучения нейронной сети и запустим процесс

In [13]:
augmentation = transforms.Compose([
    transforms.RandomRotation((-30, 30)),
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])

In [14]:
dataset = torchvision.datasets.FashionMNIST(
    root='./data', 
    download=False, 
    train=True,
    transform=augmentation
)

dataloader = torch.utils.data.DataLoader(
    dataset,
    batch_size=hyperparameters['batch_size'],
    shuffle=True,
    num_workers=hyperparameters['num_workers']
)

In [20]:
class Resnet18(nn.Module):
    def __init__(self, n_channels: int = 1, n_classes: int = 10):
        super().__init__()
        self.resnet = torchvision.models.resnet18(pretrained=False)
        # Заменим первую свертку
        self.resnet.conv1 = nn.Conv2d(in_channels=n_channels, out_channels=64, kernel_size=7, stride=2, padding=3, bias=False)
        self.resnet.fc = nn.Linear(in_features=self.resnet.fc.in_features, out_features=n_classes)
    
    def forward(self, x):
        hx = self.resnet(x)
        return hx

In [23]:
model = Resnet18(n_channels=hyperparameters['in_channels']).to(device)

In [24]:
optimizer = optim.Adam(model.parameters(), lr=hyperparameters['learning_rate'])

In [26]:
loss_fn = nn.CrossEntropyLoss()

In [28]:
training_loop(
    n_epochs = 10,
    optimizer = optimizer,
    model = model,
    loss_fn = loss_fn,
    dataloader = dataloader,
    experiment = experiment
)

  0%|          | 0/59 [00:00<?, ?it/s]

2022-02-19 07:27:57.519771 Epoch 1, Training loss 0.5988685771570368, Accuracy 0.8247833333333333


  0%|          | 0/59 [00:00<?, ?it/s]

2022-02-19 07:28:05.131737 Epoch 2, Training loss 0.40655929607860114, Accuracy 0.8408


  0%|          | 0/59 [00:00<?, ?it/s]

...

2022-02-19 07:28:30.373786 Epoch 6, Training loss 0.30487178846941154, Accuracy 0.86105


  0%|          | 0/59 [00:00<?, ?it/s]

2022-02-19 07:28:43.244678 Epoch 8, Training loss 0.28035432832725976, Accuracy 0.8845333333333333


100%|██████████| 59/59 [00:04<00:00, 11.85it/s]


2022-02-19 07:28:55.880353 Epoch 10, Training loss 0.2616798769114381, Accuracy 0.8946333333333333


**Важно после окончания эксперимента выполнить следующий вызов метода!**

In [ ]:
experiment.end()

Во время процесса обучения графики в comet.ml обновляются динамически, что позволяет нам в реальном времени следить за экспериментом (множество экспериментов на общем графике).
Рассмотри некоторые возможности которые предоставил нам comet.ml **(заметьте, мы добавили только логирование гиперпараметров, значений ошибки во время обучения и значений метрики точности)**.

1. Вкладка `Charts`

На ней мы можем сейчас наблюдать несколько графиков (comet.ml автоматически генерирует графики из имеющихся у него данных, однако мы вольны сами настроить их отображение в случае необходимости. Настроить их подписи, вид (например bar-chart вместо графиков) и т.д.).

Обратимся к рисунку.

![Charts](./imgs/charts.png)

На рисунке видно три графика, два из них идентичные (за исключением параметра сглаживания). Они отображают значения функции ошибки по мере обучения нейронной сети. И еще один график - на нем видно значения точности нейронной сети.

2. Вкладка `Panels` 

Она предназначена для настройки чего-то вроде `Dashboard` где вы можете настроить детально, какую информацию выводить на экран. На рисунке ниже можно увидеть доступные view котоыре можно добавить в panels.

![Panels](./imgs/panels.png)

3. Вкладка `Code`

После завершения эксперимента весь исполняемый код сохраняет в comet.ml. Это может быть крайне полезно для воспроизведения эксперимента. 

4. Вкладка `Metrics` 

Позволяет наблюдать за метриками в табличном представлении

![Metrics](./imgs/metrics.png)

6. Вкладка `GraphDefinition`

Позволяет взглянуть на структуру нейронной сети, в нашем случае полезной информации там не оказалось, не смотря на то что автоматическое логгирование включено. В таких случаях можно логгировать структуру сети вручную используя метод `set_model_graph()`.

7. Вкладка `Output`

Показывает вывод в потоки stdout, stderr которые были произведены во время выполнения эксперимента. Может пригодиться для логгирования ошибок.

8. Вкладка `System Metrics`

Позволяет наблюдать за потреблением ресурсов системы, на которой происходит эксперимент. Здесь можно также заметить хорошо ли нейронная сеть утилизирует возможность GPU. 

9. Вкладка `Graphics`

Данная вкладка может быть крайне полезна при работе с изображениями. Сюда можно добавлять вручную созданные (например через matplotlib) сложные графики или просто изображения (например результаты сегментации изображения). На следующем изображении указан пример, как использовалась данная возможность в одном из проектов по сегментации изображений.

![Graphics](./imgs/graphics.png)

10. Вкладка `Confusion Matrices`

В настоящем примере мы не логгировали данные матрицы, однако на рисунке ниже приведен пример того, как они отображаются. Для их логгирование в объекте эксперимента также есть специальный метод.

![ConfMatrix](./imgs/conf_matrix.png)

11. Вкладка `Histograms`

В данной вкладке можно логгировать гистограммы значений слоёв нейронной сети, Что может быть полезно при их исследовании и поиске проблем. По ним можно явно определить что происходит с весами слоёв.

12. Вкладки `HTML` и `Notes`

В данных вкладках удобно оставлять описания экспериментов. Вкладка `Notes` доступна для редактирования только с самого сайта. `HTML` же можно генерировать внутри кода эксперимента.

## Сравнение экспериментов между собой

Находясь в режиме `Panels` (находясь в режиме просмотра экспериментов проекта) можно видеть результаты всех (или выбранных по определенному фильтру или вручную) экспериментов. Там же можно вручную настроить графики если сгенерированные автоматически не позволяют произвести качественную оценку результатов.

Далее на рисунке изображено то, как может выглядть суммарный график иллюстрирующий значение метрик IoU и DiceLoss для рядя экспериментов.

![OverallBars](./imgs/overall_bars.png)

На следующем рисунке отображен график динамики обучения нейронной сети в каждом эксперименте.

![OverallLines](./imgs/overall_lines.png)

По таким графикам удобно выбирать наилучшую модель / параметры и делать какие-либо выводы.